# MR-KG Database Characteristics

This notebook provides comprehensive visualizations of the MR-KG knowledge
graph, including:
- Overall database statistics
- Trait profile similarity metrics
- Evidence profile similarity metrics
- Cross-database comparisons

**Data dependencies:** Run `just generate-all-summary-stats` before
executing this notebook.

## Setup

In [29]:
from pathlib import Path

import altair as alt
import duckdb
import pandas as pd
from yiutils.project_utils import find_project_root

# ---- Project paths ----
PROJECT_ROOT = find_project_root("docker-compose.yml")
DATA_DIR = PROJECT_ROOT / "data"
PROCESSED_DIR = DATA_DIR / "processed"
OVERALL_STATS_DIR = PROCESSED_DIR / "overall-stats"
TRAIT_ANALYSIS_DIR = PROCESSED_DIR / "trait-profiles" / "analysis"
EVIDENCE_ANALYSIS_DIR = PROCESSED_DIR / "evidence-profiles" / "analysis"

# ---- Altair configuration ----
alt.data_transformers.enable("default", max_rows=None)
alt.themes.enable("default")

print(f"Project root: {PROJECT_ROOT}")
print(f"Overall stats directory: {OVERALL_STATS_DIR}")
print(f"Trait analysis directory: {TRAIT_ANALYSIS_DIR}")
print(f"Evidence analysis directory: {EVIDENCE_ANALYSIS_DIR}")


Project root: /Users/ik18445/local-projects/+dmer/+mr-paper-data-extraction/mr-kg
Overall stats directory: /Users/ik18445/local-projects/+dmer/+mr-paper-data-extraction/mr-kg/data/processed/overall-stats
Trait analysis directory: /Users/ik18445/local-projects/+dmer/+mr-paper-data-extraction/mr-kg/data/processed/trait-profiles/analysis
Evidence analysis directory: /Users/ik18445/local-projects/+dmer/+mr-paper-data-extraction/mr-kg/data/processed/evidence-profiles/analysis


## Data Loading and Validation

In [2]:
def load_csv_with_check(filepath: Path, description: str) -> pd.DataFrame:
    """Load CSV file with existence check and error message.

    Args:
        filepath: Path to CSV file
        description: Human-readable description for error message

    Returns:
        Loaded DataFrame

    Raises:
        FileNotFoundError: If file does not exist
    """
    if not filepath.exists():
        raise FileNotFoundError(
            f"{description} not found: {filepath}\n"
            "Run 'just generate-all-summary-stats' in the processing "
            "directory first."
        )
    return pd.read_csv(filepath)


print("Loading data files...")


Loading data files...


In [3]:
# ---- Overall database statistics ----
db_summary = load_csv_with_check(
    OVERALL_STATS_DIR / "database-summary.csv", "Database summary"
)
model_stats = load_csv_with_check(
    OVERALL_STATS_DIR / "model-statistics.csv", "Model statistics"
)
journal_stats = load_csv_with_check(
    OVERALL_STATS_DIR / "journal-statistics.csv", "Journal statistics"
)

print("Loaded overall database statistics")
print(f"  - Database summary: {len(db_summary)} rows")
print(f"  - Model statistics: {len(model_stats)} models")
print(f"  - Journal statistics: {len(journal_stats)} journals")


Loaded overall database statistics
  - Database summary: 1 rows
  - Model statistics: 6 models
  - Journal statistics: 20 journals


In [4]:
# ---- Trait profile statistics ----
trait_summary = load_csv_with_check(
    TRAIT_ANALYSIS_DIR / "summary-stats-by-model.csv", "Trait profile summary"
)
trait_similarity_dist = load_csv_with_check(
    TRAIT_ANALYSIS_DIR / "similarity-distributions.csv",
    "Trait similarity distributions",
)
trait_metric_corr = load_csv_with_check(
    TRAIT_ANALYSIS_DIR / "metric-correlations.csv", "Trait metric correlations"
)
trait_count_dist = load_csv_with_check(
    TRAIT_ANALYSIS_DIR / "trait-count-distributions.csv",
    "Trait count distributions",
)

print("\nLoaded trait profile statistics")
print(f"  - Summary by model: {len(trait_summary)} models")
print(f"  - Similarity distributions: {len(trait_similarity_dist)} records")
print(f"  - Metric correlations: {len(trait_metric_corr)} records")
print(f"  - Trait count distributions: {len(trait_count_dist)} records")



Loaded trait profile statistics
  - Summary by model: 6 models
  - Similarity distributions: 6 records
  - Metric correlations: 6 records
  - Trait count distributions: 6 records


In [5]:
# ---- Evidence profile statistics ----
evidence_summary = load_csv_with_check(
    EVIDENCE_ANALYSIS_DIR / "summary-stats-by-model.csv",
    "Evidence profile summary",
)
evidence_similarity_dist = load_csv_with_check(
    EVIDENCE_ANALYSIS_DIR / "similarity-distributions.csv",
    "Evidence similarity distributions",
)
evidence_completeness = load_csv_with_check(
    EVIDENCE_ANALYSIS_DIR / "completeness-by-model.csv",
    "Evidence completeness by model",
)
evidence_matched_pairs = load_csv_with_check(
    EVIDENCE_ANALYSIS_DIR / "matched-pairs-distribution.csv",
    "Evidence matched pairs distribution",
)

print("\nLoaded evidence profile statistics")
print(f"  - Summary by model: {len(evidence_summary)} models")
print(f"  - Similarity distributions: {len(evidence_similarity_dist)} records")
print(f"  - Completeness by model: {len(evidence_completeness)} records")
print(f"  - Matched pairs: {len(evidence_matched_pairs)} records")

print("\nAll data files loaded successfully!")



Loaded evidence profile statistics
  - Summary by model: 6 models
  - Similarity distributions: 6 records
  - Completeness by model: 6 records
  - Matched pairs: 6 records

All data files loaded successfully!


---
# Section A: Overall Database Characteristics

## Plot 1: Database Entity Counts

In [6]:
# ---- Prepare data for entity counts ----
entity_data = pd.DataFrame(
    {
        "Entity": [
            "Unique Papers (PMIDs)",
            "Unique Traits",
            "Model Extraction Results",
        ],
        "Count": [
            db_summary["total_unique_pmids"].iloc[0],
            db_summary["total_unique_traits"].iloc[0],
            db_summary["total_model_results"].iloc[0],
        ],
    }
)

display(entity_data)


,Entity,Count
0,Unique Papers (PMIDs),15635
1,Unique Traits,75121
2,Model Extraction Results,50402


In [7]:
entity_chart = (
    alt.Chart(entity_data)
    .mark_bar()
    .encode(
        y=alt.Y("Entity:N", title=None, sort="-x"),
        x=alt.X("Count:Q", title="Count"),
        color=alt.Color(
            "Entity:N", legend=None, scale=alt.Scale(scheme="category10")
        ),
        tooltip=[
            alt.Tooltip("Entity:N", title="Entity"),
            alt.Tooltip("Count:Q", title="Count", format=","),
        ],
    )
    .properties(
        width=600,
        height=250,
        title="MR-KG Database Entity Counts",
    )
)

# Add text labels
text = entity_chart.mark_text(
    align="left",
    baseline="middle",
    dx=5,
).encode(text=alt.Text("Count:Q", format=","))

entity_chart + text


alt.LayerChart(...)

## Plot 2: Per-Model Extraction Statistics

In [8]:
# ---- Prepare model statistics data ----
print("Model statistics:")
display(model_stats)


Model statistics:


,model,extraction_count,unique_pmids,avg_traits_per_extraction,total_traits_extracted,avg_results_per_extraction,total_results_extracted
0,deepseek-r1-distilled,718,718,4.541193,3197.0,4.786908,3437.0
1,gpt-4-1,15626,15626,5.255483,80761.0,4.539229,70930.0
2,gpt-5,15606,15606,7.021855,106346.0,5.720877,89280.0
3,llama3,6416,6416,4.470016,28027.0,4.185474,26854.0
4,llama3-2,6670,6670,5.031044,33547.0,5.862519,39103.0
5,o4-mini,5366,5366,6.046409,31659.0,4.481178,24046.0


In [9]:
model_chart = (
    alt.Chart(model_stats)
    .mark_bar()
    .encode(
        x=alt.X("model:N", title="Model", axis=alt.Axis(labelAngle=-45)),
        y=alt.Y("extraction_count:Q", title="Extraction Count"),
        color=alt.Color(
            "model:N", legend=None, scale=alt.Scale(scheme="tableau10")
        ),
        tooltip=[
            alt.Tooltip("model:N", title="Model"),
            alt.Tooltip("extraction_count:Q", title="Extractions", format=","),
            alt.Tooltip("unique_pmids:Q", title="Unique Papers", format=","),
            alt.Tooltip(
                "avg_traits_per_extraction:Q",
                title="Avg Traits/Extraction",
                format=".2f",
            ),
        ],
    )
    .properties(
        width=600,
        height=400,
        title="Extraction Results by Model",
    )
)

model_chart


alt.Chart(...)

## Plot 3: Top Journals

In [10]:
# ---- Show top 20 journals ----
top_journals = journal_stats.head(20)

journal_chart = (
    alt.Chart(top_journals)
    .mark_bar()
    .encode(
        y=alt.Y("journal:N", title=None, sort="-x"),
        x=alt.X("paper_count:Q", title="Number of Papers"),
        color=alt.Color(
            "paper_count:Q",
            scale=alt.Scale(scheme="blues"),
            legend=None,
        ),
        tooltip=[
            alt.Tooltip("journal:N", title="Journal"),
            alt.Tooltip("paper_count:Q", title="Papers", format=","),
            alt.Tooltip("percentage:Q", title="Percentage", format=".2f"),
        ],
    )
    .properties(
        width=600,
        height=500,
        title="Top 20 Journals in MR-KG Corpus",
    )
)

journal_chart


alt.Chart(...)

---
# Section B: Trait Profile Similarity

## Plot 4: Similarity Score Distributions

In [11]:
# ---- Examine trait similarity distributions ----
print("Trait similarity distribution columns:")
print(trait_similarity_dist.columns.tolist())
print("\nFirst few rows:")
display(trait_similarity_dist.head())


Trait similarity distribution columns:
['model', 'n_pairs', 'mean_semantic_similarity', 'median_semantic_similarity', 'sd_semantic_similarity', 'p25_semantic_similarity', 'p50_semantic_similarity', 'p75_semantic_similarity', 'p95_semantic_similarity', 'mean_jaccard_similarity', 'median_jaccard_similarity', 'sd_jaccard_similarity', 'p25_jaccard_similarity', 'p50_jaccard_similarity', 'p75_jaccard_similarity', 'p95_jaccard_similarity', 'avg_query_trait_count', 'avg_similar_trait_count']

First few rows:


,model,n_pairs,mean_semantic_similarity,median_semantic_similarity,sd_semantic_similarity,p25_semantic_similarity,p50_semantic_similarity,p75_semantic_similarity,p95_semantic_similarity,mean_jaccard_similarity,median_jaccard_similarity,sd_jaccard_similarity,p25_jaccard_similarity,p50_jaccard_similarity,p75_jaccard_similarity,p95_jaccard_similarity,avg_query_trait_count,avg_similar_trait_count
0,deepseek-r1-distilled,7180,0.587806,0.604195,0.159264,0.520887,0.604195,0.685244,0.796550,0.056555,0.000000,0.094292,0.0,0.000000,0.100000,0.250000,4.452646,6.457242
1,gpt-4-1,156260,0.733955,0.748653,0.149697,0.672336,0.748653,0.821151,0.921483,0.087738,0.041667,0.132448,0.0,0.041667,0.133333,0.333333,5.168373,9.062300
2,gpt-5,156060,0.723708,0.752712,0.183608,0.668569,0.752712,0.827819,0.937146,0.095992,0.055556,0.150600,0.0,0.055556,0.125000,0.333333,6.814430,11.617711
3,llama3,64160,0.705456,0.732149,0.178862,0.644808,0.732149,0.807366,0.928961,0.101921,0.066667,0.135677,0.0,0.066667,0.157895,0.333333,4.368298,7.894451
4,llama3-2,66700,0.717829,0.730865,0.144797,0.645721,0.730865,0.806069,0.926068,0.095429,0.055556,0.138302,0.0,0.055556,0.142857,0.333333,5.029535,8.625592


In [12]:
# ---- Prepare data for layered density plot ----
# Create long format for both semantic and Jaccard similarities
semantic_data = trait_similarity_dist[
    ["model", "mean_semantic_similarity"]
].copy()
semantic_data["metric"] = "Semantic Similarity"
semantic_data.rename(
    columns={"mean_semantic_similarity": "similarity_value"},
    inplace=True,
)

jaccard_data = trait_similarity_dist[
    ["model", "mean_jaccard_similarity"]
].copy()
jaccard_data["metric"] = "Jaccard Similarity"
jaccard_data.rename(
    columns={"mean_jaccard_similarity": "similarity_value"},
    inplace=True,
)

combined_similarity = pd.concat(
    [semantic_data, jaccard_data],
    ignore_index=True,
)

print(f"Combined similarity data: {len(combined_similarity)} records")
display(combined_similarity.head())


Combined similarity data: 12 records


,model,similarity_value,metric
0,deepseek-r1-distilled,0.587806,Semantic Similarity
1,gpt-4-1,0.733955,Semantic Similarity
2,gpt-5,0.723708,Semantic Similarity
3,llama3,0.705456,Semantic Similarity
4,llama3-2,0.717829,Semantic Similarity


In [13]:
similarity_dist_chart = (
    alt.Chart(combined_similarity)
    .transform_density(
        density="similarity_value",
        as_=["similarity_value", "density"],
        groupby=["metric", "model"],
    )
    .mark_area(opacity=0.6)
    .encode(
        x=alt.X("similarity_value:Q", title="Similarity Score"),
        y=alt.Y("density:Q", title="Density"),
        color=alt.Color("metric:N", title="Metric Type"),
        facet=alt.Facet("model:N", columns=2, title="Model"),
    )
    .properties(
        width=300,
        height=200,
        title="Distribution of Trait Similarity Scores by Model",
    )
)

similarity_dist_chart


alt.Chart(...)

## Plot 5: Trait Count Distribution

In [14]:
# ---- Examine trait count distributions ----
print("Trait count distribution columns:")
print(trait_count_dist.columns.tolist())
print("\nFirst few rows:")
display(trait_count_dist.head())
print("\nSummary statistics:")
display(trait_count_dist.describe())


Trait count distribution columns:
['model', 'n_studies', 'mean_trait_count', 'median_trait_count', 'sd_trait_count', 'min_trait_count', 'max_trait_count', 'p25_trait_count', 'p75_trait_count', 'p95_trait_count']

First few rows:


,model,n_studies,mean_trait_count,median_trait_count,sd_trait_count,min_trait_count,max_trait_count,p25_trait_count,p75_trait_count,p95_trait_count
0,deepseek-r1-distilled,718,4.452646,4.0,2.858321,0,23,2.0,6.0,10.0
1,gpt-4-1,15626,5.168373,4.0,3.293611,0,38,3.0,7.0,11.0
2,gpt-5,15606,6.814430,6.0,4.594025,0,74,4.0,9.0,15.0
3,llama3,6416,4.368298,4.0,2.998381,0,31,2.0,6.0,10.0
4,llama3-2,6670,5.029535,4.0,3.213761,0,30,2.0,7.0,11.0



Summary statistics:


,n_studies,mean_trait_count,median_trait_count,sd_trait_count,min_trait_count,max_trait_count,p25_trait_count,p75_trait_count,p95_trait_count
count,6.000000,6.000000,6.00000,6.000000,6.0,6.00000,6.000000,6.000000,6.000000
mean,8400.333333,5.288868,4.50000,3.479463,0.0,39.00000,2.666667,7.166667,11.666667
std,5988.137395,0.930507,0.83666,0.656895,0.0,18.04439,0.816497,1.169045,1.966384
min,718.000000,4.368298,4.00000,2.858321,0.0,23.00000,2.000000,6.000000,10.000000
25%,5628.500000,4.596868,4.00000,3.052226,0.0,30.25000,2.000000,6.250000,10.250000
50%,6543.000000,5.098954,4.00000,3.253686,0.0,34.50000,2.500000,7.000000,11.000000
75%,13372.000000,5.717037,4.75000,3.762413,0.0,38.00000,3.000000,7.750000,12.500000
max,15626.000000,6.814430,6.00000,4.594025,0.0,74.00000,4.000000,9.000000,15.000000


In [15]:
trait_count_hist = (
    alt.Chart(trait_count_dist)
    .mark_bar()
    .encode(
        x=alt.X(
            "trait_count:Q",
            bin=alt.Bin(maxbins=30),
            title="Number of Traits per Study",
        ),
        y=alt.Y("count():Q", title="Frequency"),
        tooltip=[
            alt.Tooltip("trait_count:Q", bin=True, title="Trait Count"),
            alt.Tooltip("count():Q", title="Frequency"),
        ],
    )
    .properties(
        width=700,
        height=400,
        title="Distribution of Traits per Study",
    )
)

trait_count_hist


alt.Chart(...)

## Plot 6: Similarity Metric Correlation

In [16]:
# ---- Examine metric correlations ----
print("Metric correlation columns:")
print(trait_metric_corr.columns.tolist())
print("\nFirst few rows:")
display(trait_metric_corr.head())


Metric correlation columns:
['model', 'n_pairs', 'corr_semantic_jaccard', 'corr_semantic_query_count', 'corr_semantic_similar_count', 'corr_jaccard_query_count', 'corr_jaccard_similar_count']

First few rows:


,model,n_pairs,corr_semantic_jaccard,corr_semantic_query_count,corr_semantic_similar_count,corr_jaccard_query_count,corr_jaccard_similar_count
0,deepseek-r1-distilled,7180,0.312666,-0.081642,0.070865,-0.103874,-0.208102
1,gpt-4-1,156260,0.099284,-0.061611,0.090002,0.001824,-0.162462
2,gpt-5,156060,-0.054521,-0.044659,0.126503,-0.118533,-0.237469
3,llama3,64160,0.252862,-0.053935,0.137026,-0.015662,-0.236682
4,llama3-2,66700,0.333459,-0.264098,-0.023806,0.002194,-0.215536


In [17]:
metric_corr_scatter = (
    alt.Chart(trait_metric_corr)
    .mark_circle(size=60, opacity=0.5)
    .encode(
        x=alt.X(
            "semantic_similarity:Q",
            title="Semantic Similarity",
            scale=alt.Scale(domain=[0, 1]),
        ),
        y=alt.Y(
            "jaccard_similarity:Q",
            title="Jaccard Similarity",
            scale=alt.Scale(domain=[0, 1]),
        ),
        color=alt.Color("model:N", title="Model"),
        tooltip=[
            alt.Tooltip("model:N", title="Model"),
            alt.Tooltip(
                "semantic_similarity:Q",
                title="Semantic Similarity",
                format=".3f",
            ),
            alt.Tooltip(
                "jaccard_similarity:Q",
                title="Jaccard Similarity",
                format=".3f",
            ),
        ],
    )
    .properties(
        width=600,
        height=600,
        title="Semantic vs Jaccard Similarity Correlation",
    )
)

# Add regression line
regression = metric_corr_scatter.transform_regression(
    "semantic_similarity",
    "jaccard_similarity",
    groupby=["model"],
).mark_line()

metric_corr_scatter + regression


alt.LayerChart(...)

---
# Section C: Evidence Profile Similarity

## Plot 7: Evidence Similarity Metrics

In [32]:
# ---- Examine evidence similarity distributions ----
print("Evidence similarity distribution columns:")
print(evidence_similarity_dist.columns.tolist())
print("\nFirst few rows:")
display(evidence_similarity_dist.head())


Evidence similarity distribution columns:
['model', 'n_pairs', 'mean_direction_concordance', 'median_direction_concordance', 'sd_direction_concordance', 'p25_direction_concordance', 'p75_direction_concordance', 'p95_direction_concordance', 'n_effect_size', 'mean_effect_size_similarity', 'median_effect_size_similarity', 'sd_effect_size_similarity', 'p25_effect_size_similarity', 'p75_effect_size_similarity', 'p95_effect_size_similarity', 'n_statistical_consistency', 'mean_statistical_consistency', 'median_statistical_consistency', 'sd_statistical_consistency', 'p25_statistical_consistency', 'p75_statistical_consistency', 'p95_statistical_consistency', 'mean_evidence_overlap', 'median_evidence_overlap', 'sd_evidence_overlap', 'p25_evidence_overlap', 'p75_evidence_overlap', 'p95_evidence_overlap', 'mean_composite_equal', 'median_composite_equal', 'sd_composite_equal', 'p25_composite_equal', 'p75_composite_equal', 'p95_composite_equal', 'mean_composite_direction', 'median_composite_directio

,model,n_pairs,mean_direction_concordance,median_direction_concordance,sd_direction_concordance,p25_direction_concordance,p75_direction_concordance,p95_direction_concordance,n_effect_size,mean_effect_size_similarity,...,sd_composite_equal,p25_composite_equal,p75_composite_equal,p95_composite_equal,mean_composite_direction,median_composite_direction,sd_composite_direction,p25_composite_direction,p75_composite_direction,p95_composite_direction
0,deepseek-r1-distilled,22,0.454545,1.0,0.911685,-0.5,1.0,1.0,0,NaN,...,0.326096,0.166667,0.500,1.00,0.346562,0.230769,0.321428,0.157051,0.314103,1.000000
1,gpt-4-1,2066,0.520732,1.0,0.835880,0.0,1.0,1.0,40,0.188473,...,0.277667,0.200000,0.600,1.00,0.418880,0.384615,0.292181,0.173077,0.615385,1.000000
2,gpt-5,1249,0.413397,1.0,0.888953,-1.0,1.0,1.0,31,0.073432,...,0.245234,0.157895,0.500,0.80,0.329402,0.250000,0.260218,0.125000,0.500000,0.800000
3,llama3,608,0.449123,1.0,0.865519,-0.5,1.0,1.0,33,0.291040,...,0.313854,0.196875,0.625,1.00,0.433908,0.384615,0.328132,0.164835,0.666667,1.000000
4,llama3-2,633,0.311216,1.0,0.918216,-1.0,1.0,1.0,27,0.152230,...,0.254171,0.142857,0.500,0.86,0.328412,0.256410,0.273853,0.115385,0.500000,0.890769


In [33]:
# ---- Prepare data for multi-panel visualization ----
# Create long format for available evidence metrics (supports new and legacy names)
evidence_metrics = []

# Prefer documented summary metrics; keep legacy names for compatibility
column_mappings = [
    ("mean_direction_concordance", "Direction Concordance"),
    ("median_direction_concordance", "Direction Concordance (Median)"),
    ("mean_composite_direction", "Composite Similarity"),
    ("median_composite_direction", "Composite Similarity (Median)"),
    # Legacy per-pair metrics if present
    ("direction_concordance", "Direction Concordance"),
    ("effect_size_similarity", "Effect Size Similarity"),
    ("statistical_consistency", "Statistical Consistency"),
    ("evidence_overlap", "Evidence Overlap"),
]

for col, label in column_mappings:
    if col in evidence_similarity_dist.columns:
        metric_data = evidence_similarity_dist.loc[:, ["model", col]].dropna()
        metric_data = metric_data.rename(columns={col: "value"})
        metric_data["metric_type"] = label
        evidence_metrics.append(metric_data)

# Fallback: include any numeric mean_/median_ columns if nothing matched
if not evidence_metrics:
    numeric_cols = [
        c
        for c, dt in evidence_similarity_dist.dtypes.items()
        if c not in {"model"} and pd.api.types.is_numeric_dtype(dt)
    ]
    fallback_cols = [
        c for c in numeric_cols if c.startswith(("mean_", "median_"))
    ]
    for col in fallback_cols:
        metric_data = evidence_similarity_dist.loc[:, ["model", col]].dropna()
        metric_data = metric_data.rename(columns={col: "value"})
        label = col.replace("_", " ").title()
        metric_data["metric_type"] = label
        evidence_metrics.append(metric_data)

if evidence_metrics:
    combined_evidence = pd.concat(evidence_metrics, ignore_index=True)
    print(
        f"Combined evidence metrics: {len(combined_evidence)} records from {len(evidence_metrics)} metrics"
    )
    included = sorted({df["metric_type"].iloc[0] for df in evidence_metrics})
    print("Metrics included:", included)
    display(combined_evidence.head())
else:
    print("No evidence metrics found in the data")
    print("Available columns:", evidence_similarity_dist.columns.tolist())


Combined evidence metrics: 24 records from 4 metrics
Metrics included: ['Composite Similarity', 'Composite Similarity (Median)', 'Direction Concordance', 'Direction Concordance (Median)']


,model,value,metric_type
0,deepseek-r1-distilled,0.454545,Direction Concordance
1,gpt-4-1,0.520732,Direction Concordance
2,gpt-5,0.413397,Direction Concordance
3,llama3,0.449123,Direction Concordance
4,llama3-2,0.311216,Direction Concordance


In [36]:
if not evidence_metrics:
    print("No data available for visualization")
else:
    evidence_violin = (
        alt.Chart(combined_evidence)
        .transform_density(
            density="value",
            as_=["value", "density"],
            groupby=["metric_type", "model"],
        )
        .mark_area(orient="horizontal", opacity=0.7)
        .encode(
            x=alt.X("density:Q", title=None, stack="center", axis=None),
            y=alt.Y("value:Q", title="Metric Value"),
            color=alt.Color("model:N", title="Model"),
            column=alt.Column("metric_type:N", title="Metric Type"),
        )
        .properties(
            width=150,
            height=300,
            title="Distribution of Evidence Similarity Metrics",
        )
    )

evidence_violin


alt.Chart(...)

## Plot 8: Data Completeness by Model

In [21]:
# ---- Examine completeness data ----
print("Evidence completeness columns:")
print(evidence_completeness.columns.tolist())
print("\nData:")
display(evidence_completeness)


Evidence completeness columns:
['model', 'n_combinations', 'mean_completeness', 'median_completeness', 'sd_completeness', 'min_completeness', 'max_completeness', 'n_low', 'n_medium', 'n_high', 'prop_low', 'prop_medium', 'prop_high']

Data:


,model,n_combinations,mean_completeness,median_completeness,sd_completeness,min_completeness,max_completeness,n_low,n_medium,n_high,prop_low,prop_medium,prop_high
0,deepseek-r1-distilled,177,0.636436,0.600000,0.313231,0.111111,1.0,62.0,38.0,77.0,0.350282,0.214689,0.435028
1,gpt-4-1,1579,0.649007,0.666667,0.305837,0.100000,1.0,469.0,433.0,677.0,0.297023,0.274224,0.428752
2,gpt-5,1498,0.622519,0.555556,0.317463,0.100000,1.0,529.0,355.0,614.0,0.353138,0.236983,0.409880
3,llama3,866,0.716829,0.800000,0.309951,0.100000,1.0,205.0,188.0,473.0,0.236721,0.217090,0.546189
4,llama3-2,686,0.596243,0.500000,0.320867,0.100000,1.0,277.0,149.0,260.0,0.403790,0.217201,0.379009
5,o4-mini,1326,0.754637,1.000000,0.298903,0.100000,1.0,271.0,259.0,796.0,0.204374,0.195324,0.600302


In [22]:
# ---- Prepare data for stacked bar chart ----
# Reshape for stacking
completeness_long = pd.melt(
    evidence_completeness,
    id_vars=["model"],
    value_vars=["prop_high", "prop_medium", "prop_low"],
    var_name="completeness_category",
    value_name="proportion",
)

# Clean up category names
completeness_long["completeness_category"] = (
    completeness_long["completeness_category"]
    .str.replace("prop_", "")
    .str.title()
)

print("Completeness data (long format):")
display(completeness_long)


Completeness data (long format):


,model,completeness_category,proportion
0,deepseek-r1-distilled,High,0.435028
1,gpt-4-1,High,0.428752
2,gpt-5,High,0.409880
3,llama3,High,0.546189
4,llama3-2,High,0.379009
5,o4-mini,High,0.600302
6,deepseek-r1-distilled,Medium,0.214689
7,gpt-4-1,Medium,0.274224
8,gpt-5,Medium,0.236983
9,llama3,Medium,0.217090


In [23]:
completeness_chart = (
    alt.Chart(completeness_long)
    .mark_bar()
    .encode(
        x=alt.X("model:N", title="Model"),
        y=alt.Y("proportion:Q", title="Proportion", stack="normalize"),
        color=alt.Color(
            "completeness_category:N",
            title="Completeness",
            scale=alt.Scale(
                domain=["High", "Medium", "Low"],
                range=["#2ca02c", "#ff7f0e", "#d62728"],
            ),
        ),
        tooltip=[
            alt.Tooltip("model:N", title="Model"),
            alt.Tooltip(
                "completeness_category:N",
                title="Completeness",
            ),
            alt.Tooltip("proportion:Q", title="Proportion", format=".2%"),
        ],
    )
    .properties(
        width=600,
        height=400,
        title="Data Completeness Distribution by Model",
    )
)

completeness_chart


alt.Chart(...)

## Plot 9: Matched Pairs Distribution

In [24]:
# ---- Examine matched pairs data ----
print("Matched pairs columns:")
print(evidence_matched_pairs.columns.tolist())
print("\nFirst few rows:")
display(evidence_matched_pairs.head())


Matched pairs columns:
['model', 'n_pairs', 'mean_matched_pairs', 'median_matched_pairs', 'sd_matched_pairs', 'min_matched_pairs', 'max_matched_pairs', 'n_very_low', 'n_low', 'n_medium', 'n_high', 'prop_very_low', 'prop_low', 'prop_medium', 'prop_high', 'p25_matched_pairs', 'p50_matched_pairs', 'p75_matched_pairs', 'p95_matched_pairs']

First few rows:


,model,n_pairs,mean_matched_pairs,median_matched_pairs,sd_matched_pairs,min_matched_pairs,max_matched_pairs,n_very_low,n_low,n_medium,n_high,prop_very_low,prop_low,prop_medium,prop_high,p25_matched_pairs,p50_matched_pairs,p75_matched_pairs,p95_matched_pairs
0,deepseek-r1-distilled,22,1.000000,1.0,0.000000,1,1,22.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0,1.0,1.0,1.0,1.0
1,gpt-4-1,2066,1.166989,1.0,0.446340,1,5,2022.0,44.0,0.0,0.0,0.978703,0.021297,0.000000,0.0,1.0,1.0,1.0,2.0
2,gpt-5,1249,1.190552,1.0,0.500519,1,6,1217.0,31.0,1.0,0.0,0.974379,0.024820,0.000801,0.0,1.0,1.0,1.0,2.0
3,llama3,608,1.241776,1.0,0.628089,1,5,575.0,33.0,0.0,0.0,0.945724,0.054276,0.000000,0.0,1.0,1.0,1.0,3.0
4,llama3-2,633,1.260664,1.0,0.589988,1,5,603.0,30.0,0.0,0.0,0.952607,0.047393,0.000000,0.0,1.0,1.0,1.0,2.0


In [25]:
matched_pairs_box = (
    alt.Chart(evidence_matched_pairs)
    .mark_boxplot(extent="min-max")
    .encode(
        x=alt.X("model:N", title="Model"),
        y=alt.Y("matched_pairs:Q", title="Number of Matched Pairs"),
        color=alt.Color("model:N", legend=None),
        tooltip=[
            alt.Tooltip("model:N", title="Model"),
            alt.Tooltip("matched_pairs:Q", title="Matched Pairs"),
        ],
    )
    .properties(
        width=600,
        height=400,
        title="Distribution of Matched Trait Pairs",
    )
)

matched_pairs_box


alt.Chart(...)

---
# Section D: Cross-Database Comparison

## Plot 10: Model Coverage Comparison

In [26]:
# ---- Prepare coverage comparison data ----
trait_coverage = trait_summary[["model", "total_combinations"]].copy()
trait_coverage["database"] = "Trait Profiles"
trait_coverage.rename(
    columns={"total_combinations": "combinations"},
    inplace=True,
)

evidence_coverage = evidence_summary[["model", "total_combinations"]].copy()
evidence_coverage["database"] = "Evidence Profiles"
evidence_coverage.rename(
    columns={"total_combinations": "combinations"},
    inplace=True,
)

combined_coverage = pd.concat(
    [trait_coverage, evidence_coverage],
    ignore_index=True,
)

print("Coverage comparison data:")
display(combined_coverage)


Coverage comparison data:


,model,combinations,database
0,deepseek-r1-distilled,718,Trait Profiles
1,gpt-4-1,15626,Trait Profiles
2,gpt-5,15606,Trait Profiles
3,llama3,6416,Trait Profiles
4,llama3-2,6670,Trait Profiles
5,o4-mini,5366,Trait Profiles
6,deepseek-r1-distilled,177,Evidence Profiles
7,gpt-4-1,1579,Evidence Profiles
8,gpt-5,1498,Evidence Profiles
9,llama3,866,Evidence Profiles


In [27]:
coverage_chart = (
    alt.Chart(combined_coverage)
    .mark_bar()
    .encode(
        x=alt.X("model:N", title="Model", axis=alt.Axis(labelAngle=-45)),
        y=alt.Y("combinations:Q", title="Number of Combinations"),
        color=alt.Color("database:N", title="Database Type"),
        column=alt.Column("database:N", title=None),
        tooltip=[
            alt.Tooltip("model:N", title="Model"),
            alt.Tooltip("database:N", title="Database"),
            alt.Tooltip("combinations:Q", title="Combinations", format=","),
        ],
    )
    .properties(
        width=300,
        height=400,
        title="Trait vs Evidence Profile Coverage by Model",
    )
)

coverage_chart


alt.Chart(...)

---
# Summary

## Key Insights

Based on the visualizations above:

**Literature Coverage:**
- Total papers in the corpus
- Publication year range
- Growth trajectory over time

**Trait Diversity:**
- Number of unique traits extracted
- Distribution of traits per study
- Variation across extraction models

**Model Performance:**
- Extraction quality varies by model
- Semantic and Jaccard similarities show correlation
- Different models have different coverage patterns

**Similarity Patterns:**
- Trait profiles show distinct similarity distributions
- Evidence profiles are sparser with more variability
- Cross-database patterns reveal different aspects of MR studies

## Data Quality Notes

- **Evidence profiles** have sparser data due to requirement for
  quantitative results
- **Trait profiles** cover all studies with extracted traits
- **Completeness** varies by model and reporting standards in the
  literature
- **Matched pairs** depend on trait overlap between studies

## Next Steps

- See `docs/processing/summary-statistics.md` for detailed methodology
- Analysis scripts available in `processing/scripts/analysis/`
- Export visualizations using the cells below for manuscript inclusion

## Export Visualizations (Optional)

Uncomment and run the cells below to save plots for manuscript use.

In [ ]:
# # ---- Create output directory ----
# output_dir = PROCESSED_DIR / "figures" / "mr-kg"
# output_dir.mkdir(parents=True, exist_ok=True)
# print(f"Output directory: {output_dir}")


In [ ]:
# # ---- Save all plots as JSON ----
# entity_chart.save(str(output_dir / "entity_counts.json"))
# model_chart.save(str(output_dir / "model_statistics.json"))
# journal_chart.save(str(output_dir / "top_journals.json"))
# similarity_dist_chart.save(str(output_dir / "trait_similarity_distributions.json"))
# trait_count_hist.save(str(output_dir / "trait_count_distribution.json"))
# (metric_corr_scatter + regression).save(str(output_dir / "metric_correlation.json"))
# if evidence_metrics:
#     evidence_violin.save(str(output_dir / "evidence_similarity_metrics.json"))
# completeness_chart.save(str(output_dir / "completeness_by_model.json"))
# matched_pairs_box.save(str(output_dir / "matched_pairs_distribution.json"))
# coverage_chart.save(str(output_dir / "coverage_comparison.json"))
# print("All plots saved successfully!")


In [31]:
# Save evidence violin chart JSON for manuscript use
output_dir = PROCESSED_DIR / "figures" / "mr-kg"
output_dir.mkdir(parents=True, exist_ok=True)
print(f"Output directory: {output_dir}")

try:
    evidence_violin.save(str(output_dir / "evidence_similarity_metrics.json"))
    print("Saved:", output_dir / "evidence_similarity_metrics.json")
except NameError:
    print("evidence_violin not defined. Run the evidence plot cell first.")


Output directory: /Users/ik18445/local-projects/+dmer/+mr-paper-data-extraction/mr-kg/data/processed/figures/mr-kg
Saved: /Users/ik18445/local-projects/+dmer/+mr-paper-data-extraction/mr-kg/data/processed/figures/mr-kg/evidence_similarity_metrics.json
